In [1]:
import numpy as np
import pandas as pd
import os
import utils
import gmm
import datetime
import run_all

In [2]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data'
fig_path = curr_dir + '/../figs'
results_path = curr_dir + '/../results'
animation_path = curr_dir + '/../animation'
belltown_path = data_path + '/Belltown_Hour'
commcore_path = data_path + '/CommercialCore_Hour'
pikepine_path = data_path + '/PikePine_Hour'
dennytriangle_path = data_path + '/DennyTriangle_Hour'
firsthill_path = data_path + '/FirstHill_Hour'
raw_transaction_path = data_path + '/RawTransactionData'

In [3]:
block_info = pd.read_csv(os.path.join(data_path, 'block_info.csv'))
block_info = block_info.loc[block_info['PaidParkingArea'] == 'Belltown']
block_keys = block_info['ElementKey'].unique().tolist()

paystation_info = pd.read_csv(os.path.join(data_path, 'paystation_info.csv'))
paystation_info = paystation_info.loc[paystation_info['PAIDAREA'] == 'Belltown']
pay_keys = paystation_info['ELMNTKEY'].unique().tolist()

In [4]:
element_keys, loads, gps_loc, park_data, N, P, idx_to_day_hour, day_hour_to_idx = utils.load_data_new(data_path, [belltown_path], verbose=False)

In [6]:
time = 58
num_comps = 4
run_all.run_figures(element_keys, loads, gps_loc, num_comps, data_path, fig_path, animation_path, time, False)

In [ ]:
run_all.gmm_simulations(park_data, gps_loc, times=[50,51,58], k=5, p_value=.01, fig_path=fig_path, results_path=results_path)

In [ ]:
results = gmm.locational_demand_analysis(park_data, gps_loc, k=5, p_value=.01)

In [ ]:
paystation_info.head()

In [ ]:
results = gmm.locational_demand_analysis(park_data, gps_loc)
days = [result[0] for result in results]
hours = [result[1] for result in results]
errors = [result[2] for result in results]
morans_mixture = [result[3] for result in results]
morans_adjacent = [result[4] for result in results]
means = [result[5] for result in results]